In [35]:
import os
import sys; sys.path.append("~/HHbbgg_ETH_devel/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)
import preprocessing_utils as preprocessing
reload(preprocessing)
import plotting_utils as plotting
reload(plotting)
import optimization_utils as optimization
reload(optimization)
import postprocessing_utils as postprocessing
reload(postprocessing)
import pandas as pd
import root_pandas as rpd
import matplotlib.pyplot as plt
import json
from ROOT import TLorentzVector
from optparse import OptionParser, make_option
from  pprint import pprint
import commands
import time
import datetime
start_time = time.time()

In [36]:
filedir = '/work/nchernya/HHbbgg_ETH_devel/outfiles/20190321_2017_2016_for_optimization/'
filename = 'Total_preselection_diffNaming_2016_2017_21_03_2019_transformedMVA.root'

sig_names = ['reducedTree_sig','reducedTree_sig_2017']
bkg_names = ['reducedTree_bkg_0','reducedTree_bkg_1','reducedTree_bkg_2','reducedTree_bkg_0_2017']
signal_list = []
bkg_list = []
lumi_ratio = 41.5/35.9
columns = ['MX',"Mjj","event","weight",'overlapSave','MVAOutputTransformed','Mgg']

for sig in sig_names:
    df = rpd.read_root(filedir+filename,sig, columns = columns ).query('weight>=0.')
    if '2017' in sig:
        df['weight']*=lumi_ratio
    signal_list.append(df)
for bkg in bkg_names:
    df = rpd.read_root(filedir+filename,bkg, columns =  columns).query('weight>=0.')
    if '2017' in bkg:
        df['weight']*=lumi_ratio 
    df['weight'] = df['weight']*df['overlapSave']
    bkg_list.append(df)  
    
sig = pd.concat(signal_list, axis=0, ignore_index=True)
bkg = pd.concat(bkg_list, axis=0, ignore_index=True)

In [37]:
training = '/work/nchernya/HHbbgg_ETH_devel/Training/output_files/training_with_29_05_2019_classification.pkl'
branch_names = 'MX,MVAOutputTransformed'.split(",") 
branch_names = [c.strip() for c in branch_names]


from sklearn.externals import joblib
import xgboost as xgb
n_threads=20
loaded_model = joblib.load(training)
loaded_model._Booster.set_param('nthread', 20)
Y_pred_sig = loaded_model.predict_proba(sig[branch_names].values)[:,loaded_model.n_classes_-1].astype(np.float64)
Y_pred_bkg = loaded_model.predict_proba(bkg[branch_names].values)[:,loaded_model.n_classes_-1].astype(np.float64)


In [46]:
print max(Y_pred_sig)
print min(Y_pred_bkg)


0.9923577904701233
0.008672451600432396


In [18]:
def stackFeatures(df,names):
    vec = []
    dictVar = {}
    i=0
    for feat in names:
        vec.append(np.asarray(df[feat]))
        dictVar[feat] = i
        i+=1
    totalVec = []
    for i in range(len(vec)):
        if i == 0:
            totalVec = vec[i]
        else:
            totalVec = np.column_stack((totalVec,vec[i]))


    return totalVec,dictVar


In [19]:
from root_numpy import array2root
def saveTree(processPath,dictVar,vector,MVAVector,nameTree="reducedTree"):
    i=0
    for key in dictVar.keys():
        if i == 0:
            writeMode='recreate'
            i=1
        else:
            writeMode='update'

        v=(np.asarray(vector[:,dictVar[key]]))
        name = key   
        v.dtype = [(name, np.float64)]
        array2root(v, processPath, nameTree, mode = writeMode)

    v=(np.asarray(MVAVector.ravel()))
    v.dtype = [('ClassificationBDT', np.float64)]
    array2root(v, processPath, nameTree, mode ='update')

In [20]:
event_branches = ['event','weight','Mgg']

processPath=filedir+'Total_preselection_2016_2017_21_03_2019_classificationBDT_signal.root'
nTot_sig,dictVar_sig = stackFeatures(sig,branch_names+event_branches)
saveTree(processPath,dictVar_sig,nTot_sig,Y_pred_sig,'reducedTree_sig')

processPath=filedir+'Total_preselection_2016_2017_21_03_2019_classificationBDT_bkg.root'
nTot_bkg,dictVar_bkg = stackFeatures(bkg,branch_names+event_branches)
saveTree(processPath,dictVar_bkg,nTot_bkg,Y_pred_bkg,'reducedTree_bkg')

